This notebook generates the designs from "[Dispersion-engineered metasurfaces reaching broadband 90% relative diffraction efficiency](https://www.nature.com/articles/s41467-023-38185-2)" by Chen et al.

The detailed layout and dimensions for the meta-atoms are in the [supplementary material](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-023-38185-2/MediaObjects/41467_2023_38185_MOESM1_ESM.pdf).

In [ ]:
import numpy as onp
import matplotlib.pyplot as plt
from skimage import measure

def plus_nanostructure(x1, x2, y1, y2, dim):
    """Return density array for plus-shaped nanostructures."""
    x, y = onp.meshgrid(
        onp.arange(-dim // 2, dim // 2) + 0.5,
        onp.arange(-dim // 2, dim // 2) + 0.5,
        indexing="ij",
    )
    p = onp.ones(x.shape, dtype=bool)
    p &= (x < x1 / 2) & (x > -x1 / 2)
    p &= (y < y1 / 2) & (y > -y1 / 2)
    p &= (x < x2 / 2) & (x > -x2 / 2) | ((y < y2 / 2) & (y > -y2 / 2))
    return p.astype(float)

def ibeam_nanostructure(x1, x2, y1, y2, dim, rotate):
    """Return density array for ibeam-shaped nanostructures."""
    x, y = onp.meshgrid(
        onp.arange(-dim // 2, dim // 2) + 0.5,
        onp.arange(-dim // 2, dim // 2) + 0.5,
        indexing="ij",
    )
    p = onp.ones(x.shape, dtype=bool)
    p &= (x < x1 / 2) & (x > -x1 / 2)
    p &= (y < y1 / 2) & (y > -y1 / 2)
    p &= (x > x2 / 2) | (x < -x2 / 2) | (y < (y1 / 2 - y2)) & (y > -(y1 / 2 - y2))
    if rotate:
        p = onp.rot90(p)
    return p.astype(float)

# Create the density arrays for the 8 meta-atoms from the library. Values are
# taken from figure S3 of Chen et al.
nanostructures = (
    plus_nanostructure(x1=156, x2=86, y1=140, y2=86, dim=400),
    plus_nanostructure(x1=230, x2=60, y1=220, y2=60, dim=400),
    ibeam_nanostructure(y1=200, y2=60, x1=200, x2=80, dim=400, rotate=False),
    ibeam_nanostructure(y1=320, y2=120, x1=180, x2=60, dim=400, rotate=True),
    ibeam_nanostructure(y1=340, y2=120, x1=180, x2=60, dim=400, rotate=True),
    ibeam_nanostructure(y1=340, y2=130, x1=280, x2=80, dim=400, rotate=True),
    ibeam_nanostructure(y1=320, y2=110, x1=300, x2=60, dim=400, rotate=True),
    ibeam_nanostructure(y1=340, y2=90, x1=300, x2=60, dim=400, rotate=True),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(2, 4, figsize=(8, 4))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.set_xticks([])
    ax.set_yticks([])
plt.subplots_adjust(hspace=0.1, wspace=0.1)

In [ ]:
import jax
import jax.numpy as jnp
from invrs_gym.challenges import meta_atom_library
from invrs_gym.utils import transforms

challenge = meta_atom_library()
params = challenge.component.init(jax.random.PRNGKey(0))

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

In [ ]:
from totypes import json_utils

with open("library1.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))